In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


In [19]:
def get_elastic_force(edges, positions, k):
    elastic_force = np.zeros_like(positions)
    for i, j in edges:
        elastic_force[i] += k * (positions[j] - positions[i])
        elastic_force[j] += k * (positions[i] - positions[j])
    return elastic_force

def get_electric_force(positions, electric_charge):
    electric_force = np.zeros_like(positions)
    for i, pos_i in enumerate(positions):
        for j, pos_j in enumerate(positions):
            if i != j:
                diff = pos_i - pos_j
                norm = np.linalg.norm(diff)
                electric_force[i] += electric_charge**2 * diff / norm**3
    return electric_force

def get_force(edges, positions, k, electric_charge):
    elastic_force = get_elastic_force(edges, positions, k)
    electric_force = get_electric_force(positions, electric_charge)
    return elastic_force + electric_force

def update_positions(edges, positions, k, electric_charge, learning_rate):
    force = get_force(edges, positions, k, electric_charge)
    positions += learning_rate * force
    return positions

def draw_graph(edges, positions):
    for i, j in edges:
        plt.plot([positions[i, 0], positions[j, 0]], [positions[i, 1], positions[j, 1]], color='black')
    plt.scatter(positions[:, 0], positions[:, 1], color='black')
    plt.show()

def get_lines(edges, positions):
    x, y = [], []
    for i, j in edges:
        x.append([positions[i, 0], positions[j, 0]])
        y.append([positions[i, 1], positions[j, 1]])
    return x, y

def get_degrees(edges):
    degrees = np.zeros(np.max(edges) + 1)
    for i, j in edges:
        degrees[i] += 1
        degrees[j] += 1
    return degrees


In [18]:
edges = np.array([[0, 1], [0, 2], [0, 3], [0, 4], [1, 2]])
np.max(edges)

4

In [30]:
def create_animation(edges, k, electric_charge, num_iterations, animation_name, limits=(-0.5, 1.5), learning_rate=0.01):

    memory = []
    nodes = list(set(v for edge in edges for v in edge))
    positions = np.random.rand(len(nodes), 2)
    degrees = get_degrees(edges)

    for i in range(num_iterations):
        positions = update_positions(edges, positions, k, electric_charge, learning_rate)
        memory.append(positions.copy())

    # Create a figure for the animation
    fig, ax = plt.subplots()
    ax.set_aspect('equal')

    # Initialize an empty graph for the animation
    line, = ax.plot([], [], color='black', lw=1)
    scatter = ax.scatter([], [], color='blue', s=100)
    # set limits
    ax.set_xlim(limits)
    ax.set_ylim(limits)

    # Define the initialization function for the animation
    def init():
        line.set_data([], [])
        scatter.set_offsets(np.array([]).reshape(0, 2))  # Corrected the initialization of empty array
        return line, scatter


    # Define the animation update function
    def update(i):
        positions = memory[i]
        x, y = positions.T
        x2, y2 = get_lines(edges, positions)
        line.set_xdata(x2)
        line.set_ydata(y2)
        data = np.hstack((x.reshape(-1, 1), y.reshape(-1, 1)))
        # colors from colormap proportional to degree
        scatter.set_color(plt.cm.OrRd(degrees / np.max(degrees)))
        scatter.set_sizes(100 * degrees)
        scatter.set_offsets(data)

        return line, scatter

    # Create the animation
    ani = FuncAnimation(fig, update, init_func=init, frames=num_iterations, blit=True, repeat=False)

    # Show the animation
    plt.show()

    # Save the animation
    ani.save(animation_name, writer='imagemagick', fps=60)


# Full graph

In [31]:
k = 1.0
electric_charge = 1.0
num_iterations = 100

In [ ]:
# Define your graph and initial positions here
edges = np.array([(0, 1), (1, 2), (0, 2), (0, 3), (1, 3), (2, 3), (0, 4), (1, 4), (2, 4), (3, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6)])
create_animation(edges, k, electric_charge, num_iterations, 'full_graph.gif', limits=(-1, 2))

# Random Graph

In [ ]:
num_nodes = 40
edge_probability = 0.25
edges_random_graph = []

for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        if random.random() < edge_probability:
            edges_random_graph.append([i, j])

edges_random_graph = np.array(edges_random_graph)
create_animation(edges_random_graph, k, electric_charge, num_iterations=200, animation_name='random_graph2.gif', learning_rate=0.01, limits=(-10, 10))


# Lattice

In [ ]:
# edges for lattice graph
edges = np.array([(0, 1), (1, 2), (2, 3), (0, 4), (1, 5), (2, 6), (3, 7), (4, 5), (5, 6), (6, 7), (4, 8), (8, 9), (5, 9), (9, 10), (6, 10), (10, 11), (7, 11)])
create_animation(edges, k, electric_charge, num_iterations=200, animation_name='lattice_graph.gif', learning_rate=0.06, limits=(-5, 5))


# Chain

In [ ]:
edges = np.array([(i, i+1) for i in range(10)])
create_animation(edges, k, electric_charge, num_iterations=200, animation_name='chain_graph.gif', learning_rate=0.06, limits=(-10, 10))